# Tornado Synthetic Dataset - 3D Steady
> [cgl datasets](https://cgl.ethz.ch/research/visualization/data.php)
> Steady means no time is involved

In [1]:
import numpy as np
import pyvista as pv
np.set_printoptions(precision=3)

In [2]:
file = '..\\Data\\tornado3d.vti'
mesh = pv.read(file)
print(mesh)

UniformGrid (0x22b481aafa0)
  N Cells:	2048383
  N Points:	2097152
  X Bounds:	-1.000e+01, 1.000e+01
  Y Bounds:	-1.000e+01, 1.000e+01
  Z Bounds:	-1.000e+01, 1.000e+01
  Dimensions:	128, 128, 128
  Spacing:	1.575e-01, 1.575e-01, 1.575e-01
  N Arrays:	3



In [23]:
# Get the magnitude from the vector data
w,v,u = mesh.point_data['w'], mesh.point_data['v'], mesh.point_data['u']
mag = (np.sqrt(u**2 + v**2 + w**2))

# vectors = np.vstack((w,v,u)).T

vectors = np.empty((mesh.n_points, 3))
vectors[:,0] = u
vectors[:,1] = v
vectors[:,2] = w

print(f'mesh dimensions : {mesh.dimensions}')
print(f'128 * 128 * 128 = {128*128*128}')
print(f'u shape : {u.shape}\nv shape : {v.shape}\nw shape : {w.shape}')
print(f'vectors shape : {vectors.shape}')

mesh dimensions : (128, 128, 128)
128 * 128 * 128 = 2097152
u shape : (2097152,)
v shape : (2097152,)
w shape : (2097152,)
vectors shape : (2097152, 3)


In [26]:
# https://docs.pyvista.org/api/core/_autosummary/pyvista.RectilinearGrid.streamlines.html

mesh['vectors'] = vectors

boundary = mesh.decimate_boundary().extract_all_edges()

# Sphere Source
stream, src = mesh.streamlines(
    'vectors', 
    return_source=True,
    terminal_speed=0,
    n_points=10,
    source_radius=1,
    source_center=(0,0,-9)
)

# Line Source
# stream, src = mesh.streamlines(
#     'vectors', 
#     return_source=True,
#     terminal_speed=0,
#     n_points=10,
#     pointa=(-10, -10, -10),
#     pointb=(10, 10, -10),
#     progress_bar=True
# )

sargs=dict(vertical=True, title_font_size=16)
p = pv.Plotter()
p.add_mesh(stream.tube(radius=0.02), lighting=False, scalar_bar_args=sargs)
p.add_mesh(src)
p.add_mesh(boundary, color='grey', opacity=.25)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [27]:
print(stream.point_data)

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : vectors
Active Vectors  : Normals
Active Texture  : None
Active Normals  : None
Contains arrays :
    u                       float32  (1572,)
    v                       float32  (1572,)
    w                       float32  (1572,)
    vectors                 float64  (1572, 3)            SCALARS
    IntegrationTime         float64  (1572,)
    Vorticity               float64  (1572, 3)
    Rotation                float64  (1572,)
    AngularVelocity         float64  (1572,)
    Normals                 float64  (1572, 3)            VECTORS


In [29]:
vec = stream.point_data['vectors']
print(vec[1,2] - vec[0,2])
print(vec[2,2] - vec[1,2])

-0.002725528545454159
-0.002728434598950724


In [30]:
print(vectors[1,2] - vectors[0,2])
print(vectors[2,2] - vectors[1,2])

-0.0005564773455262184
-0.0005454383790493011
